<a href="https://colab.research.google.com/github/mohandabdelhalim99/Deep_learning_practice/blob/main/Resent(implemention).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Layer

In [4]:
class Identity_Block(tf.keras.Model):

  def __init__(self,filters,kernel_size):
    super(Identity_Block,self).__init__()
    
    self.conv1=tf.keras.layers.Conv2D(filters,kernel_size,padding='same')
    self.bn1=tf.keras.layers.BatchNormalization()

    self.conv2=tf.keras.layers.Conv2D(filters,kernel_size,padding='same')
    self.bn2=tf.keras.layers.BatchNormalization()

    self.act=tf.keras.layers.Activation('relu')
    self.add=tf.keras.layers.Add()


  def call(self,input_tensor):
    x=self.conv1(input_tensor)
    x=self.bn1(x)
    x=self.act(x)
    x=self.conv2(x)
    x=self.bn2(x)
    x=self.add([x,input_tensor])
    x=self.act(x)
    return x




In [8]:
class Resnet(tf.keras.Model):

  def __init__(self,numclasses):
    super(Resnet,self).__init__()
    self.conv=tf.keras.layers.Conv2D(64,(7,7),activation='relu')
    self.bn=tf.keras.layers.BatchNormalization()
    self.maxpool=tf.keras.layers.MaxPool2D((3,3))
    self.id1=Identity_Block(64,3)
    self.id2=Identity_Block(64,3)
    self.avgpool=tf.keras.layers.GlobalAveragePooling2D()
    self.classify=tf.keras.layers.Dense(numclasses,activation='softmax')


  def call(self,inputs):
    x=self.conv(inputs)
    x=self.bn(x)
    x=self.maxpool(x)
    x=self.id1(x)
    x=self.id2(x)
    x=self.avgpool(x)
    return self.classify(x)

In [9]:
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']


resnet = Resnet(10)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')
dataset = dataset.map(preprocess).batch(32)


resnet.fit(dataset, epochs=1)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to ./data/mnist/3.0.1. Subsequent calls will reuse this data.
1875/1875 [==============================] - 160s 85ms/step - loss: 0.1129 - accuracy: 0.9692
